# Housing Data Workflow Notebook

Modular workflow where you can run individual steps independently.
Run cells in order or skip any steps you don't need.

Each step shows dataframe views and statistics for inspection.

## Quick Start
- Run **Setup** cell first
- Then run any combination of Step 1-4 cells
- Skip cells you don't want to execute
- Each cell is self-contained and shows results

## 🔧 Setup

Run this cell first to import modules and define helper functions.

In [ ]:
import sys
from pathlib import Path
from typing import Optional

import pandas as pd
import os
import warnings
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patches as mpatches

# Add current directory to path for local imports
sys.path.append(".")

# Import our workflow modules
from fetch_affordable_housing_data import update_local_data, verify_and_fetch_hpd_data, verify_and_fetch_hpd_projects_data
from query_dob_filings import query_dob_bisweb_bin, query_dob_bisweb_bbl, query_dobnow_bin, query_dobnow_bbl, decompose_bbl, query_condo_lots_for_bbl, query_dob_by_address, pad_block, pad_lot
from query_co_filings import query_co_api, DOB_NOW_CO_URL, DOB_CO_URL

print("✅ All imports successful")

# Helper functions
def _normalize_bin(bin_value) -> Optional[str]:
    """Normalize BIN to a clean string."""
    if pd.isna(bin_value):
        return None
    try:
        return str(int(float(bin_value)))
    except (TypeError, ValueError):
        value = str(bin_value).strip()
        return value or None

def _extract_bins_from_df(df: pd.DataFrame) -> list[str]:
    """Extract BINs from a DataFrame and return as a list for CO searches."""
    candidate_cols = []
    for col in df.columns:
        if col.lower() in ("bin", "bin_normalized"):
            candidate_cols.append(col)
    if not candidate_cols:
        raise SystemExit(f"Could not find a BIN column in DataFrame")

    bins = []
    for val in df[candidate_cols[0]].dropna():
        normalized = _normalize_bin(val)
        if normalized:
            bins.append(normalized)
    
    # Remove duplicates using set, then sort
    unique_bins = set()
    for b in bins:
        if b:
            unique_bins.add(b)
    return sorted(unique_bins)

def _query_co_filings_from_bins_and_bbls(bin_list: list[str], bbl_list: list[str] = None, output_path: Path = None) -> pd.DataFrame:
    """Query CO filings using a list of BINs and BBLs (no file needed)."""
    # Convert BINs to integers for API query
    bin_ints = []
    for bin_str in bin_list:
        if str(bin_str).isdigit():
            bin_ints.append(int(bin_str))
    bins = sorted(list(set(bin_ints)))
    
    # Query DOB NOW Certificate of Occupancy API
    print("=" * 70)
    print("QUERYING DOB NOW CERTIFICATE OF OCCUPANCY")
    print("=" * 70)
    dob_now_co = query_co_api(DOB_NOW_CO_URL, bins, bin_column="bin")
    
    # Query DOB Certificate Of Occupancy API
    print("\n" + "=" * 70)
    print("QUERYING DOB CERTIFICATE OF OCCUPANCY")
    print("=" * 70)
    dob_co = query_co_api(DOB_CO_URL, bins, bin_column="bin_number")
    
    # Query by BBL for records not found by BIN
    dob_now_co_bbl = pd.DataFrame()
    dob_co_bbl = pd.DataFrame()
    
    if bbl_list and len(bbl_list) > 0:
        # Convert BBLs to integers for API query
        bbl_ints = []
        for bbl_str in bbl_list:
            if str(bbl_str).isdigit():
                bbl_ints.append(int(bbl_str))
        bbls = sorted(list(set(bbl_ints)))
        
        if len(bbls) > 0:
            # Query DOB NOW CO by BBL
            print("\n" + "=" * 70)
            print("QUERYING DOB NOW CERTIFICATE OF OCCUPANCY BY BBL")
            print("=" * 70)
            dob_now_co_bbl = query_co_api(DOB_NOW_CO_URL, bbls, bin_column="bbl")
            
            # Query DOB CO by BBL
            print("\n" + "=" * 70)
            print("QUERYING DOB CERTIFICATE OF OCCUPANCY BY BBL")
            print("=" * 70)
            dob_co_bbl = query_co_api(DOB_CO_URL, bbls, bin_column="bbl")
            
            # Add source and normalize columns
            if not dob_now_co_bbl.empty:
                print(f"\nDOB NOW CO Filings (by BBL): {len(dob_now_co_bbl)} records")
                dob_now_co_bbl['source'] = 'DOB_NOW_CO_BBL'
                if 'bbl' in dob_now_co_bbl.columns:
                    dob_now_co_bbl['bbl_normalized'] = dob_now_co_bbl['bbl'].astype(str).str.zfill(10)
            
            if not dob_co_bbl.empty:
                print(f"\nDOB CO Filings (by BBL): {len(dob_co_bbl)} records")
                dob_co_bbl['source'] = 'DOB_CO_BBL'
                if 'bbl' in dob_co_bbl.columns:
                    dob_co_bbl['bbl_normalized'] = dob_co_bbl['bbl'].astype(str).str.zfill(10)
    
    # Combine results
    print("\n" + "=" * 70)
    print("SUMMARY")
    print("=" * 70)
    
    if not dob_now_co.empty:
        print(f"\nDOB NOW CO Filings: {len(dob_now_co)} records")
        dob_now_co['source'] = 'DOB_NOW_CO'
        if 'bin' in dob_now_co.columns:
            dob_now_co['bin_normalized'] = dob_now_co['bin'].astype(str)
    
    if not dob_co.empty:
        print(f"\nDOB CO Filings: {len(dob_co)} records")
        dob_co['source'] = 'DOB_CO'
        if 'bin_number' in dob_co.columns:
            dob_co['bin_normalized'] = dob_co['bin_number'].astype(str)
    
    # Combine all dataframes (BIN and BBL results)
    dfs_to_combine = []
    if not dob_now_co.empty:
        dfs_to_combine.append(dob_now_co)
    if not dob_co.empty:
        dfs_to_combine.append(dob_co)
    if not dob_now_co_bbl.empty:
        dfs_to_combine.append(dob_now_co_bbl)
    if not dob_co_bbl.empty:
        dfs_to_combine.append(dob_co_bbl)
    
    if len(dfs_to_combine) >= 2:
        all_cols = list(set([col for df in dfs_to_combine for col in df.columns]))
        if 'bin_normalized' not in all_cols:
            all_cols.append('bin_normalized')
        if 'source' not in all_cols:
            all_cols.append('source')
        dob_now_co_aligned = dob_now_co.reindex(columns=all_cols)
        dob_co_aligned = dob_co.reindex(columns=all_cols)
        combined = pd.concat([dob_now_co_aligned, dob_co_aligned], ignore_index=True)
    elif not dob_now_co.empty:
        combined = dob_now_co.copy()
    elif not dob_co.empty:
        combined = dob_co.copy()
    else:
        print("\nNo CO filings found in either API")
        return pd.DataFrame()
    
    print(f"\nTotal combined records: {len(combined)}")
    
    return combined

print("✅ Helper functions defined")

✅ All imports successful
✅ Helper functions defined


## 📥 Step 1: Fetch HPD Data

Load or refresh the HPD affordable housing dataset.

**Options:**
- Set `refresh_data = True` to fetch fresh data
- Set `refresh_data = False` to use existing data

In [ ]:
# Step 1 Configuration
refresh_data = False  # Set to True to fetch fresh HPD data
hpd_output_path = "data/raw/Affordable_Housing_Production_by_Building.csv"  # Output path for HPD data
refresh_hpd_projects = False  # Set to True to fetch fresh HPD projects data

print("=" * 70)
print("STEP 1: FETCH HPD DATA")
print("=" * 70)

# Start quality tracking

# Handle HPD projects cache refresh if requested
if refresh_hpd_projects:
    print("Force refreshing HPD projects cache...")
    hpd_projects_df, hpd_projects_path = verify_and_fetch_hpd_projects_data(use_existing=False)
    print(f"HPD projects cache refreshed: {len(hpd_projects_df)} records\n")

if refresh_data:
    print("Fetching fresh HPD data from NYC Open Data...")
    hpd_df, hpd_csv = update_local_data(hpd_output_path)
else:
    print("Verifying local HPD data against API...")
    hpd_df, hpd_csv = verify_and_fetch_hpd_data(output_path=hpd_output_path, use_projects_cache=not refresh_hpd_projects)

if not hpd_csv.exists():
    raise SystemExit(f"HPD dataset not found at {hpd_csv}")

# Get total units before filter
original_count = len(hpd_df)
original_units = hpd_df['Total Units'].sum()

# Filter to New Construction only
hpd_new_construction_df = hpd_df[hpd_df["Reporting Construction Type"] == "New Construction"].copy()

filtered_count = len(hpd_new_construction_df)
filtered_units = hpd_new_construction_df['Total Units'].sum()
filtered_out = original_count - filtered_count
filtered_units_out = original_units - filtered_units

print(f"🏗️ Filtered to New Construction only:")
print(f"  Original: {original_count:,} projects, {original_units:,} total units")
print(f"  Filtered: {filtered_count:,} projects ({filtered_count/original_count*100:.1f}%), {filtered_units:,} total units ({filtered_units/original_units*100:.1f}%)")
print(f"  Removed: {filtered_out:,} non-new construction projects ({filtered_out/original_count*100:.1f}%), {filtered_units_out:,} units filtered out ({filtered_units_out/original_units*100:.1f}%)")

print(f"✅ Step 1 complete: {len(hpd_new_construction_df):,} records loaded")
print(f"📁 Data location: {hpd_csv}")

# Display the dataframe
print("\n🔍 HPD Dataset Overview (New Construction only):")
print(f"Shape: {hpd_new_construction_df.shape}")
print("\nColumns:")
for col in hpd_new_construction_df.columns:
    print(f"  - {col}")

print("\n📊 Sample Data:")
display(hpd_new_construction_df.head())
print("\n📈 Basic Statistics:")
display(hpd_new_construction_df.describe(include="all"))

STEP 1: FETCH HPD DATA
Verifying local HPD data against API...
STEP 1: VERIFY AND FETCH HPD DATA
Local HPD data file not found at data/raw/Affordable_Housing_Production_by_Building.csv
Fetching fresh data from API...
Fetching affordable housing data from NYC Open Data API...
Endpoint: https://data.cityofnewyork.us/resource/hg8x-zxpr.json
Fetching records 1-1000...
  Retrieved 1000 records (total: 1,000)
Fetching records 1001-2000...
  Retrieved 1000 records (total: 2,000)
Fetching records 2001-3000...
  Retrieved 1000 records (total: 3,000)
Fetching records 3001-4000...
  Retrieved 1000 records (total: 4,000)
Fetching records 4001-5000...
  Retrieved 1000 records (total: 5,000)
Fetching records 5001-6000...
  Retrieved 1000 records (total: 6,000)
Fetching records 6001-7000...
  Retrieved 1000 records (total: 7,000)
Fetching records 7001-8000...
  Retrieved 1000 records (total: 8,000)
Fetching records 8001-9000...
  Retrieved 604 records (total: 8,604)
Fetching records 8605-9604...

Com

,Project ID,Project Name,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,...,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,project_Program Group,project_Extended Affordability Status,project_Prevailing Wage Status,project_Planned Tax Benefit
0,44218,MEC E. 125TH ST. PARCEL B WEST,2018-12-31T00:00:00.000,None,987329,2319,3 AVENUE,Manhattan,10035,1017907501,...,NaN,NaN,297.0,NaN,297.0,404,None,None,None,None
1,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,None,927737,335,RALPH AVENUE,Brooklyn,11233,3015560003,...,NaN,NaN,NaN,13.0,13.0,13,None,None,None,None
2,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,None,969695,35,ROCHESTER AVENUE,Brooklyn,11233,3017090009,...,NaN,NaN,NaN,8.0,8.0,8,None,None,None,None
3,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,None,975702,18-22,SUYDAM PLACE,Brooklyn,11233,3017090028,...,NaN,NaN,NaN,15.0,15.0,15,None,None,None,None
4,44223,ROCHESTER SUYDAM PHASE 1,2021-06-30T00:00:00.000,None,977564,329,RALPH AVENUE,Brooklyn,11233,3015560007,...,NaN,NaN,NaN,10.0,10.0,10,None,None,None,None



📈 Basic Statistics:


,Project ID,Project Name,Project Start Date,Project Completion Date,Building ID,Number,Street,Borough,Postcode,BBL,...,6-BR+ Units,Unknown-BR Units,Counted Rental Units,Counted Homeownership Units,All Counted Units,Total Units,project_Program Group,project_Extended Affordability Status,project_Prevailing Wage Status,project_Planned Tax Benefit
count,4216,4216,4216,0,3184,4216,4216,4216,3182,3115,...,12.0,106.000000,3029.000000,1276.000000,4206.000000,4216.000000,0,0,0,0
unique,3749,2718,1850,0,3162,1965,1048,5,146,3007,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0
top,53017,CONFIDENTIAL,2016-06-27T00:00:00.000,NaN,975696,----,----,Brooklyn,11239,2035150020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,83,1032,84,NaN,2,1032,1032,1873,165,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.688679,37.684714,1.639498,27.636472,52.016129,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,6.993008,67.426568,4.116363,59.488316,107.460375,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,3.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,9.000000,1.000000,5.000000,10.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,1.000000,38.000000,1.000000,20.000000,50.000000,NaN,NaN,NaN,NaN


In [4]:
# How many unique counts are there by project id as primary key per program group,
# and show total units in parentheticals (but NOT for the unique project counts).

# Compute total units per Program Group (all rows)
units_per_group = hpd_new_construction_df.groupby('Program Group')['Total Units'].sum()

print("Program Group counts (raw rows) (total units in parentheses):")
raw_row_counts = hpd_new_construction_df['Program Group'].value_counts()
for group, count in raw_row_counts.items():
    units = units_per_group.get(group, 0)
    print(f"{group}: {count} rows ({units} units)")
print()

# Group by Program Group, count unique Project IDs
unique_proj_counts = hpd_new_construction_df.groupby('Program Group')['Project ID'].nunique().sort_values(ascending=False)
unique_proj_ids = (
    hpd_new_construction_df
    .groupby('Program Group')
    .apply(lambda df: df['Project ID'].unique())
)

print("Program Group counts (unique Project ID as primary key):")
for group, count in unique_proj_counts.items():
    print(f"{group}: {count} projects")
print()

print("\nTax Abatement by Program Group (based on unique Project ID):")
if 'Planned Tax Benefit' in hpd_new_construction_df.columns:
    # For this, deduplicate by Project ID first
    unique_project_rows = hpd_new_construction_df.drop_duplicates(subset=['Project ID'])
    tax_abate_ct = (
        unique_project_rows
        .groupby('Program Group')['Planned Tax Benefit']
        .value_counts(dropna=False)
        .unstack(fill_value=0)
        .sort_index(axis=1)
    )
    # Also display total units per Program Group in this table, if desired
    units_per_group_project = unique_project_rows.groupby('Program Group')['Total Units'].sum()
    print("Total units (unique Project ID per Program Group):")
    display(units_per_group_project)
    display(tax_abate_ct)
else:
    print("Column 'Planned Tax Benefit' not found in dataset.")

# Make a version of this with unit count by program and tax benefit
if 'Planned Tax Benefit' in unique_project_rows.columns and 'Program Group' in unique_project_rows.columns:
    units_pivot = (
        unique_project_rows
        .groupby(['Program Group', 'Planned Tax Benefit'])['Total Units']
        .sum()
        .unstack(fill_value=0)
        .sort_index(axis=1)
    )
    print("Total units by Program Group and Planned Tax Benefit (unique Project ID only):")
    display(units_pivot)
else:
    print("Required columns not found for unit pivot table.")

# Calculate average units per year by Program Group and Planned Tax Benefit

if 'Project Start Date' in unique_project_rows.columns and 'Total Units' in unique_project_rows.columns:
    # Extract year from 'Project Start Date'
    unique_project_rows = unique_project_rows.copy()
    unique_project_rows['Project Year'] = pd.to_datetime(unique_project_rows['Project Start Date'], errors='coerce').dt.year

    avg_units_per_year = (
        unique_project_rows
        .groupby(['Program Group', 'Planned Tax Benefit', 'Project Year'])['Total Units']
        .sum()
        .reset_index()
    )

    # Now calculate the average units per year by program group and tax abatement
    avg_units_table = (
        avg_units_per_year
        .groupby(['Program Group', 'Planned Tax Benefit'])['Total Units']
        .mean()
        .unstack(fill_value=0)
        .sort_index(axis=1)
    )
    print("Average units per year by Program Group and Planned Tax Benefit (unique Project ID only):")
    display(avg_units_table)
else:
    print("Required columns not found for average units per year table.")



KeyError: 'Program Group'

In [ ]:
# We'll use the full raw HPD data, because we want all programs, not just Multifamily Finance Program
if 'Project Start Date' in hpd_new_construction_df.columns and 'Total Units' in hpd_new_construction_df.columns:
    hpd_bar_df = hpd_new_construction_df.copy()
    hpd_bar_df['Project Year'] = pd.to_datetime(hpd_bar_df['Project Start Date'], errors='coerce').dt.year

    # Only focus on desired groups
    programs_of_interest = ['Multifamily Finance Program', 'Multifamily Incentives Program']
    mask = hpd_bar_df['Program Group'].isin(programs_of_interest)
    hpd_bar_df = hpd_bar_df[mask & hpd_bar_df['Project Year'].notna()]

    # Fill NAs in Planned Tax Benefit with "None"
    hpd_bar_df['Planned Tax Benefit'] = hpd_bar_df['Planned Tax Benefit'].fillna('None')

    # Prepare for grouped bar with stack
    # Pivot: rows = Project Year, columns = (Program Group, Planned Tax Benefit), values = sum of units
    pivot = (
        hpd_bar_df
        .groupby(['Project Year', 'Program Group', 'Planned Tax Benefit'])['Total Units']
        .sum()
        .reset_index()
    )

    # Ensure proper order of years and programs
    years = sorted(pivot['Project Year'].dropna().unique())
    tax_benefits = sorted(pivot['Planned Tax Benefit'].unique())
    # Keep consistent order for bars
    program_order = ['Multifamily Finance Program', 'Multifamily Incentives Program']

    # Prepare data structure: for each year, for each program, get breakdown by tax benefit
    bar_data = {}
    for year in years:
        bar_data[year] = {}
        for prog in program_order:
            mask = (pivot['Project Year'] == year) & (pivot['Program Group'] == prog)
            year_prog_data = pivot[mask].set_index('Planned Tax Benefit')['Total Units'].reindex(tax_benefits, fill_value=0)
            bar_data[year][prog] = year_prog_data.values

    # Number of bars per group (2 programs), group by year, stacked by tax benefit
    x = range(len(years))
    width = 0.35

    fig, ax = plt.subplots(figsize=(14, 7))

    # Colors for planned tax benefits
    color_map = cm.get_cmap('tab20', len(tax_benefits))
    colors = []
    for i in range(len(tax_benefits)):
        colors.append(color_map(i))

    bottoms_p1 = [0] * len(years)
    bottoms_p2 = [0] * len(years)

    # For each tax benefit, draw the stack pieces for both programs
    legend_handles = []
    for idx, tax in enumerate(tax_benefits):
        values_p1 = []
        for year in years:
            values_p1.append(bar_data[year][program_order[0]][idx])
        values_p2 = []
        for year in years:
            values_p2.append(bar_data[year][program_order[1]][idx])

        x_positions_p1 = []
        for i in x:
            x_positions_p1.append(i - width/2)
        x_positions_p2 = []
        for i in x:
            x_positions_p2.append(i + width/2)
        
        legend_labels = []
        for h in legend_handles:
            legend_labels.append(h.get_label())
        
        bar1 = ax.bar(
            x_positions_p1, values_p1, width,
            bottom=bottoms_p1, color=colors[idx],
            label=tax if (tax not in legend_labels) else None,
            edgecolor='black', hatch='////'
        )
        bar2 = ax.bar(
            x_positions_p2, values_p2, width,
            bottom=bottoms_p2, color=colors[idx],
            label=None,
            edgecolor='black'
        )

        legend_labels = []
        for h in legend_handles:
            legend_labels.append(h.get_label())
        if tax not in legend_labels:
            legend_handles.append(bar1)

        new_bottoms_p1 = []
        for b, v in zip(bottoms_p1, values_p1):
            new_bottoms_p1.append(b + v)
        bottoms_p1 = new_bottoms_p1
        
        new_bottoms_p2 = []
        for b, v in zip(bottoms_p2, values_p2):
            new_bottoms_p2.append(b + v)
        bottoms_p2 = new_bottoms_p2

    # Add year labels
    ax.set_xticks(x)
    year_labels = []
    for y in years:
        year_labels.append(str(int(y)))
    ax.set_xticklabels(year_labels, rotation=45)
    ax.set_xlabel("Project Start Year")
    ax.set_ylabel("Total Units Financed")
    ax.set_title("Units Financed by Year: Multifamily Finance and Incentives Programs\nColored by Planned Tax Benefit")

    # Custom legend for program groups
    progs = [
        mpatches.Patch(color='gray', label='Multifamily Finance Program', ec='black', hatch='////'),
        mpatches.Patch(color='gray', label='Multifamily Incentives Program', ec='black')
    ]
    # Only add one legend for planned tax benefit
    handles_tax = []
    for i in range(len(tax_benefits)):
        handles_tax.append(plt.Rectangle((0,0),1,1, color=colors[i], edgecolor='black', label=f"{tax_benefits[i]}"))
    legend1 = ax.legend(handles=handles_tax, title="Planned Tax Benefit", loc='upper right')
    ax.add_artist(legend1)
    # Add manual tick legend for program bars
    bar_locs = [x[0] - width/2, x[0] + width/2]
    ax.bar(bar_locs[0], 0, width, color='white', hatch='////', ec='black', label='Multifamily Finance Program')
    ax.bar(bar_locs[1], 0, width, color='white', ec='black', label='Multifamily Incentives Program')
    ax.legend(
        handles=[
            plt.Rectangle((0,0),1,1, facecolor='white', hatch='////', edgecolor='black', label='Multifamily Finance Program'),
            plt.Rectangle((0,0),1,1, facecolor='white', edgecolor='black', label='Multifamily Incentives Program')
        ], title="Program Group", loc='upper left'
    )

    ax.grid(True, which='major', axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
else:
    print("Required columns ('Project Start Date', 'Total Units') not found in HPD Data.")



In [ ]:
# Count and sample: Planned Tax Benefit '421a' and Project Start Date in 2025
if "Planned Tax Benefit" in hpd_new_construction_df.columns and "Project Start Date" in hpd_new_construction_df.columns:
    # Filter for 421a and 2025 start year
    mask_421a_2025 = (
        (hpd_new_construction_df["Planned Tax Benefit"] == "421a") &
        (hpd_new_construction_df["Project Start Date"].astype(str).str.startswith("2025"))
    )
    hpd_421a_2025_df = hpd_new_construction_df[mask_421a_2025]

    # Count unique projects (by Project ID), and total units
    total_projects = hpd_421a_2025_df["Project ID"].nunique() if "Project ID" in hpd_421a_2025_df.columns else len(hpd_421a_2025_df)
    total_units = hpd_421a_2025_df["Total Units"].sum() if "Total Units" in hpd_421a_2025_df.columns else "N/A"

    print(f"Total projects with Planned Tax Benefit '421a' and 2025 Start Date: {total_projects:,}")
    print(f"Total units in these projects: {total_units:,}")

    # Show up to 5 sample rows
    pd.set_option('display.max_columns', None)
    print("\nSample 421a Planned Tax Benefit projects with Project Start Date in 2025:")
    display(hpd_421a_2025_df)
    pd.reset_option('display.max_columns')
else:
    print("One or both of the columns 'Planned Tax Benefit' or 'Project Start Date' not found in HPD DataFrame.")



In [ ]:
# Filter to Multifamily Finance Program (already filtered to New Construction in Step 1)
original_count = len(hpd_new_construction_df)
print(f"Using HPD data from Step 1: {original_count:,} total buildings (New Construction)")

# Apply filters: Multifamily Finance Program (already filtered to New Construction in Step 1)
hpd_multifamily_finance_new_construction_df = hpd_new_construction_df[
    hpd_new_construction_df["Program Group"] == "Multifamily Finance Program"
].copy()
filtered_count = len(hpd_multifamily_finance_new_construction_df)

print(f"🏗️ Filtered to Multifamily Finance Program:")
print(f"  Original: {original_count:,} buildings (New Construction)")
print(f"  Filtered: {filtered_count:,} buildings ({filtered_count/original_count*100:.1f}%)")
print(f"📁 Created critical DataFrame in memory: {filtered_count:,} Multifamily Finance Program (New Construction) buildings")


## 🏗️ Step 3: Query DOB Filings

Search for DOB New Building filings.


In [ ]:
# Step 3A: BIN/BBL Prep and Filtering

# Use hpd_multifamily_finance_new_construction_df from Step 2.5 (in memory)
print(f"Using filtered dataset from Step 2.5: {len(hpd_multifamily_finance_new_construction_df):,} Multifamily Finance Program (New Construction) buildings")

# Extract BINs and BBLs from the filtered data
bins = []
bin_counts = hpd_multifamily_finance_new_construction_df['BIN'].value_counts()
unique_bins = bin_counts[bin_counts == 1].index.tolist()
# Extract BBLs properly using decompose_bbl function

bbls = []
for idx, row in hpd_multifamily_finance_new_construction_df.iterrows():
    if pd.notna(row.get("BBL")):
        bbl_result = decompose_bbl(str(row["BBL"]))
        if bbl_result and len(bbl_result) >= 3:
            borough, block, lot = bbl_result
            bbls.append((borough, block, lot))


# Filter out bad/placeholder BINs (e.g., 1000000, 2000000, 3000000, 4000000, 5000000)
# These are placeholder values that don't exist in DOB
def is_bad_bin(bin_str):
    """Check if BIN is a placeholder/bad value."""
    if not bin_str or pd.isna(bin_str) or str(bin_str).lower() == 'nan':
        return True
    bin_str_clean = str(bin_str).strip()
    # Check for pattern: [1-5]000000 (borough placeholder BINs)
    if bin_str_clean in ['1000000', '2000000', '3000000', '4000000', '5000000']:
        return True
    return False

if 'BIN' in hpd_multifamily_finance_new_construction_df.columns:
    bins = []
for b in hpd_multifamily_finance_new_construction_df['BIN'].dropna():
        b_str = str(b)
        if b_str != 'nan':
            b_clean = b_str.replace('.0', '')
            if not is_bad_bin(b_clean) and b_clean in unique_bins:
                bins.append(b_clean)

if 'BBL' in hpd_multifamily_finance_new_construction_df.columns:
    bbl_col = hpd_multifamily_finance_new_construction_df['BBL'].astype(str).str.zfill(10)
    bbls = []
    for bbl_val in bbl_col:
        if len(bbl_val) == 10:
            bbl_tuple = (
                bbl_val[0],                     # borough code (as string)
                bbl_val[1:6],                   # block (padded 5 chars)
                bbl_val[6:]                     # lot   (padded 4 chars)
            )
            bbls.append(bbl_tuple)

print(f"\n📋 Prepared {len(bins)} BINs and {len(bbls)} BBLs for DOB queries")


In [ ]:
# 1. BISWEB BIN for all buildings

print("BISWEB BIN QUERY (ALL BUILDINGS)")
print("=" * 70)
print(f"▶️ Querying BISWEB BIN for {len(bins)} buildings...")
dob_bisweb_bin_df = query_dob_bisweb_bin(bins)
bisweb_bin_matches = set()
if not dob_bisweb_bin_df.empty and "bin__" in dob_bisweb_bin_df.columns:
    bisweb_bin_matches = set(dob_bisweb_bin_df["bin__"].dropna().astype(str).unique())
bisweb_bin_unmatched = []
for b in bins:
    if b not in bisweb_bin_matches:
        bisweb_bin_unmatched.append(b)
print(f"BISWEB BIN: {len(bisweb_bin_matches)} matches, {len(bisweb_bin_unmatched)} need BBL fallback")




# Preview BISWEB BIN results
if not dob_bisweb_bin_df.empty:
    print("\n📊 BISWEB BIN sample:")
    display(dob_bisweb_bin_df.head())
else:
    print("\n⚠️ No BISWEB BIN results")

In [ ]:
# 2. DOB NOW BIN for all buildings
print("=" * 70)
print("DOB NOW BIN QUERY (ALL BUILDINGS)")
print("=" * 70)
print(f"▶️ Querying DOB NOW BIN for {len(bins)} buildings...")
dob_now_bin_df = query_dobnow_bin(bins)
dobnow_bin_matches = set()
if not dob_now_bin_df.empty:
    dobnow_bin_matches = set(dob_now_bin_df["bin"].dropna().astype(str).unique())
    print("📊 DOB NOW BIN sample:")
    display(dob_now_bin_df.head())
else:
    print("⚠️ No DOB NOW BIN results")

print(f"DOB NOW BIN: {len(dobnow_bin_matches)} unique BINs and {len(dob_now_bin_df)} total job records")


In [ ]:
# 3. DETERMINE UNMATCHED PROJECTS

# Combine matched BINs from both API queries
all_matched_bins = bisweb_bin_matches.union(dobnow_bin_matches)

# Find projects with BINs that failed both BISWEB and DOB NOW queries
# Create a DataFrame of projects whose BINs were not matched by either BISWEB or DOB NOW queries.
# This uses .isin(all_matched_bins) to check which projects' BINs are missing from the set of matched BINs,
# and selects only those projects to proceed to BBL fallback lookup.
unmatched_projects_df = hpd_multifamily_finance_new_construction_df[
    ~hpd_multifamily_finance_new_construction_df["BIN"]
        .astype(str)
        .str.replace(".0", "")
        .isin(all_matched_bins)
].copy()

print(f"BIN QUERY RESULTS SUMMARY")
print(f"=" * 50)
print(f"Total projects in dataset: {len(hpd_multifamily_finance_new_construction_df)}")
matched_projects = hpd_multifamily_finance_new_construction_df[hpd_multifamily_finance_new_construction_df["BIN"].astype(str).str.replace(".0", "").isin(all_matched_bins)]
print(f"Unique BINs found in BISWEB/DOB NOW APIs: {len(all_matched_bins)}")
print(f"Projects covered by those BINs: {len(matched_projects)} (some BINs cover multiple projects)")

# Analyze BIN sharing
bin_counts = hpd_multifamily_finance_new_construction_df["BIN"].value_counts()
bins_used_multiple_times = len(bin_counts[bin_counts > 1])
extra_projects_from_sharing = bin_counts[bin_counts > 1].sum() - bins_used_multiple_times
print(f"Projects needing fallback queries: {len(unmatched_projects_df)}")
print(f"BIN matching success rate: {len(matched_projects)/len(hpd_multifamily_finance_new_construction_df)*100:.1f}% of projects covered")

# Sanity check
total_accounted_for = len(matched_projects) + len(unmatched_projects_df)
print(f"Sanity check: {len(matched_projects)} + {len(unmatched_projects_df)} = {total_accounted_for} (should equal {len(hpd_multifamily_finance_new_construction_df)})")

if not unmatched_projects_df.empty:
    print(f"Unmatched projects will proceed to fallback queries (BBL → Condo → Address)")
else:
    print(f"All projects successfully matched via BIN queries! No fallbacks needed.")


In [ ]:
hpd_multifamily_finance_new_construction_df[
    (hpd_multifamily_finance_new_construction_df["BIN"].duplicated(keep=False)) & 
    (~hpd_multifamily_finance_new_construction_df["BIN"].astype(str).apply(is_bad_bin))
]

In [ ]:
# BISWEB BBL FALLBACK
if not unmatched_projects_df.empty:
    print("" + "=" * 70)
    print("BISWEB BBL FALLBACK")
    print("=" * 70)
    
    # Extract BBLs from unmatched projects
    bbl_tuples = []
    for _, row in unmatched_projects_df.iterrows():
        if pd.notna(row.get("BBL")):
            bbl_result = decompose_bbl(str(row["BBL"]))
            if bbl_result and len(bbl_result) >= 3:
                bbl_tuples.append(bbl_result)
    
    # Deduplicate BBLs
    bbl_tuples = list(set(bbl_tuples))
    print(f"Extracted {len(bbl_tuples)} unique BBLs from unmatched projects")
    
    # Query BISWEB BBL API
    print(f"▶️ Querying BISWEB BBL for {len(bbl_tuples)} BBLs...")
    dob_bisweb_bbl_df = query_dob_bisweb_bbl(bbl_tuples)
    
    # DOB NOW BBL fallback
    print("" + "=" * 70)
    print("DOB NOW BBL FALLBACK")
    print("=" * 70)
    dob_now_bbl_df = query_dobnow_bbl(bbl_tuples)
else:
    # No unmatched projects, initialize empty dataframes
    dob_bisweb_bbl_df = pd.DataFrame()
    dob_now_bbl_df = pd.DataFrame()


In [ ]:
# For Multifamily Finance Program (MFP) new construction projects, find those with no DOB match in any table.

# Use hpd_multifamily_finance_new_construction_df from Step 3A (already filtered to MFP new construction)
hpd_multifamily_finance_new_construction_for_matching_df = hpd_multifamily_finance_new_construction_df.copy()

# Defensive: set of unique Project IDs for matching
mfp_project_ids = set(hpd_multifamily_finance_new_construction_for_matching_df['Project ID'].unique())

# Combine all DOB dataframes and normalize BIN columns
all_dob_dfs = []

# Normalize BIN columns in each DOB dataframe
if not dob_bisweb_bin_df.empty:
    if 'bin__' in dob_bisweb_bin_df.columns:
        dob_bisweb_bin_df = dob_bisweb_bin_df.copy()
        dob_bisweb_bin_df['bin_normalized'] = dob_bisweb_bin_df['bin__'].astype(str).str.replace('.0', '')
    # Ensure BBL is displayed as a string, not float
    if 'bbl' in dob_bisweb_bin_df.columns:
        dob_bisweb_bin_df['bbl'] = dob_bisweb_bin_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    all_dob_dfs.append(dob_bisweb_bin_df)

if not dob_bisweb_bbl_df.empty:
    if 'bin__' in dob_bisweb_bbl_df.columns:
        dob_bisweb_bbl_df = dob_bisweb_bbl_df.copy()
        dob_bisweb_bbl_df['bin_normalized'] = dob_bisweb_bbl_df['bin__'].astype(str).str.replace('.0', '')
    elif 'bin' in dob_bisweb_bbl_df.columns:
        dob_bisweb_bbl_df = dob_bisweb_bbl_df.copy()
        dob_bisweb_bbl_df['bin_normalized'] = dob_bisweb_bbl_df['bin'].astype(str).str.replace('.0', '')
    # Ensure BBL is displayed as a string, not float
    if 'bbl' in dob_bisweb_bbl_df.columns:
        dob_bisweb_bbl_df['bbl'] = dob_bisweb_bbl_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    all_dob_dfs.append(dob_bisweb_bbl_df)

if not dob_now_bin_df.empty:
    if 'bin' in dob_now_bin_df.columns:
        dob_now_bin_df = dob_now_bin_df.copy()
        dob_now_bin_df['bin_normalized'] = dob_now_bin_df['bin'].astype(str).str.replace('.0', '')
    # Ensure BBL is displayed as a string, not float
    if 'bbl' in dob_now_bin_df.columns:
        dob_now_bin_df['bbl'] = dob_now_bin_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    all_dob_dfs.append(dob_now_bin_df)

if not dob_now_bbl_df.empty:
    if 'bin' in dob_now_bbl_df.columns:
        dob_now_bbl_df = dob_now_bbl_df.copy()
        dob_now_bbl_df['bin_normalized'] = dob_now_bbl_df['bin'].astype(str).str.replace('.0', '')
    # Ensure BBL is displayed as a string, not float
    if 'bbl' in dob_now_bbl_df.columns:
        dob_now_bbl_df['bbl'] = dob_now_bbl_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    all_dob_dfs.append(dob_now_bbl_df)

# Use combined_dob_df from Step 3A if available, otherwise combine here
if 'combined_dob_df' in globals() and not combined_dob_df.empty:
    combined_dob_with_normalized_bbl_df = combined_dob_df.copy()
    # Ensure bin_normalized is created from bin__ or bin column
    if 'bin_normalized' not in combined_dob_with_normalized_bbl_df.columns:
        if 'bin__' in combined_dob_with_normalized_bbl_df.columns:
            combined_dob_with_normalized_bbl_df['bin_normalized'] = combined_dob_with_normalized_bbl_df['bin__'].astype(str).str.replace('.0', '', regex=False)
        elif 'bin' in combined_dob_with_normalized_bbl_df.columns:
            combined_dob_with_normalized_bbl_df['bin_normalized'] = combined_dob_with_normalized_bbl_df['bin'].astype(str).str.replace('.0', '', regex=False)
    # Ensure bbl_normalized is created
    if 'bbl_normalized' not in combined_dob_with_normalized_bbl_df.columns:
        if 'bbl' in combined_dob_with_normalized_bbl_df.columns:
            combined_dob_with_normalized_bbl_df['bbl_normalized'] = combined_dob_with_normalized_bbl_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    print(f'Total DOB records (from Step 3A): {len(combined_dob_with_normalized_bbl_df)}')
elif all_dob_dfs:
    combined_dob_with_normalized_bbl_df = pd.concat(all_dob_dfs, ignore_index=True)
    print(f'Total DOB records: {len(combined_dob_with_normalized_bbl_df)}')
else:
    combined_dob_with_normalized_bbl_df = pd.DataFrame()
    print('No DOB records found')

# Prepare HPD data for matching - normalize BIN and ensure BBL is string
hpd_multifamily_finance_new_construction_with_normalized_ids_df = hpd_multifamily_finance_new_construction_for_matching_df.copy()
hpd_multifamily_finance_new_construction_with_normalized_ids_df['bin_normalized'] = hpd_multifamily_finance_new_construction_with_normalized_ids_df['BIN'].astype(str).str.replace('.0', '')
hpd_multifamily_finance_new_construction_with_normalized_ids_df['bbl_normalized'] = hpd_multifamily_finance_new_construction_with_normalized_ids_df['BBL'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)

# Join on BIN first
if not combined_dob_with_normalized_bbl_df.empty and 'bin_normalized' in combined_dob_with_normalized_bbl_df.columns:
    hpd_matched_on_bin_df = pd.merge(
        hpd_multifamily_finance_new_construction_with_normalized_ids_df,
        combined_dob_with_normalized_bbl_df[['bin_normalized']].drop_duplicates(),
        on='bin_normalized',
        how='inner'
    )
    matched_project_ids_bin = set(hpd_matched_on_bin_df['Project ID'].unique())
    print(f'Projects matched on BIN: {len(matched_project_ids_bin)}')
else:
    matched_project_ids_bin = set()

# Join on BBL for those that didn't match on BIN
hpd_unmatched_on_bin_df = hpd_multifamily_finance_new_construction_with_normalized_ids_df[~hpd_multifamily_finance_new_construction_with_normalized_ids_df['Project ID'].isin(matched_project_ids_bin)]

# Initialize BBL matching result
matched_project_ids_bbl = set()

# Reconstruct BBL in DOB data for sources that don't have it (like BISWEB)
# Reconstruct BBL from borough, block, lot for records that don't have it
def reconstruct_bbl(row):
    if pd.isna(row.get('borough')) or pd.isna(row.get('block')) or pd.isna(row.get('lot')):
        return None
    borough_map = {'MANHATTAN': '1', 'BRONX': '2', 'BROOKLYN': '3', 'QUEENS': '4', 'STATEN ISLAND': '5'}
    borough_code = borough_map.get(str(row['borough']).upper(), None)
    if not borough_code:
        return None
    # Remove leading zeros from block/lot for BBL reconstruction
    block_str = str(int(float(str(row['block']).replace('.0', ''))))
    lot_str = str(int(float(str(row['lot']).replace('.0', ''))))
    # Reconstruct: borough(1) + block(5) + lot(4) = 10 digits
    bbl_str = borough_code + block_str.zfill(5) + lot_str.zfill(4)
    return bbl_str.zfill(10)

# Always reconstruct BBL for records that need it (BISWEB data doesn't have bbl column)
combined_dob_with_normalized_bbl_df['bbl_reconstructed'] = combined_dob_with_normalized_bbl_df.apply(reconstruct_bbl, axis=1)
# Normalize BBL in DOB data (use existing bbl or reconstructed)
# Use bbl column if available, otherwise use reconstructed BBL
if 'bbl' in combined_dob_with_normalized_bbl_df.columns:
    # Use existing bbl column, normalized to 10 digits
    combined_dob_with_normalized_bbl_df['bbl_normalized'] = combined_dob_with_normalized_bbl_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    # Fill missing values with reconstructed BBL
    if 'bbl_reconstructed' in combined_dob_with_normalized_bbl_df.columns:
        combined_dob_with_normalized_bbl_df['bbl_normalized'] = combined_dob_with_normalized_bbl_df['bbl_normalized'].fillna(combined_dob_with_normalized_bbl_df['bbl_reconstructed'])
elif 'bbl_reconstructed' in combined_dob_with_normalized_bbl_df.columns:
    # Use reconstructed BBL if no bbl column exists
    combined_dob_with_normalized_bbl_df['bbl_normalized'] = combined_dob_with_normalized_bbl_df['bbl_reconstructed']
else:
    combined_dob_with_normalized_bbl_df['bbl_normalized'] = None

# Now match on BBL
if 'bbl_normalized' in combined_dob_with_normalized_bbl_df.columns and combined_dob_with_normalized_bbl_df['bbl_normalized'].notna().any():
    hpd_matched_on_bbl_df = pd.merge(
        hpd_unmatched_on_bin_df,
        combined_dob_with_normalized_bbl_df[['bbl_normalized']].drop_duplicates(),
        on='bbl_normalized',
        how='inner'
    )
    matched_project_ids_bbl = set(hpd_matched_on_bbl_df['Project ID'].unique())
    print(f'Projects matched on BBL (fallback): {len(matched_project_ids_bbl)}')
else:
    matched_project_ids_bbl = set()
    print('No BBL data available for matching')

# Combine all matched project IDs
dob_matched_project_ids = matched_project_ids_bin | matched_project_ids_bbl

# Find projects without DOB matches
mfp_projects_without_dob = mfp_project_ids - dob_matched_project_ids

print(f'\nTotal Multifamily Finance Program new construction projects: {len(mfp_project_ids)}')
print(f'Projects with DOB matches: {len(dob_matched_project_ids)}')
print(f'Number of these with NO DOB row in any table: {len(mfp_projects_without_dob)}')

# Debug: show a sample of matched and unmatched projects
if len(matched_project_ids_bin) > 0:
    print(f'\nSample matched on BIN: {list(matched_project_ids_bin)[:3]}')
if len(matched_project_ids_bbl) > 0:
    print(f'Sample matched on BBL: {list(matched_project_ids_bbl)[:3]}')
if len(mfp_projects_without_dob) > 0:
    print(f'Sample unmatched: {list(mfp_projects_without_dob)[:3]}')

# DEBUG: Analyze a sample project to understand matching
if len(mfp_projects_without_dob) > 0:
    sample_project_id = list(mfp_projects_without_dob)[0]
    sample_project = hpd_multifamily_finance_new_construction_for_matching_df[hpd_multifamily_finance_new_construction_for_matching_df['Project ID'] == sample_project_id]
    print(f'\n=== DEBUG: Sample unmatched project ===')
    print(f'Project ID: {sample_project_id}')
    print(f'Number of buildings in project: {len(sample_project)}')
    sample_bins = sample_project['BIN'].dropna().astype(str).str.replace('.0', '').tolist()
    sample_bbls = sample_project['BBL'].dropna().apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None).tolist()
    print(f'BINs in project: {sample_bins[:5]}')
    print(f'BBLs in project: {sample_bbls[:5]}')
    
    # Check if these BINs/BBLs exist in DOB data
    if not combined_dob_with_normalized_bbl_df.empty:
        if 'bin_normalized' in combined_dob_with_normalized_bbl_df.columns:
            dob_bins = set(combined_dob_with_normalized_bbl_df['bin_normalized'].dropna().astype(str).unique())
            matching_bins = []
            for b in sample_bins:
                if b in dob_bins:
                    matching_bins.append(b)
            print(f'BINs found in DOB data: {matching_bins[:5] if matching_bins else "None"}')
        if 'bbl' in combined_dob_with_normalized_bbl_df.columns:
            dob_bbls = set(combined_dob_with_normalized_bbl_df['bbl'].dropna().apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None).unique())
            matching_bbls = []
            for b in sample_bbls:
                if b in dob_bbls:
                    matching_bbls.append(b)
            print(f'BBLs found in DOB data: {matching_bbls[:5] if matching_bbls else "None"}')
# Show the head of the table of unmatched projects (project-level)
if len(mfp_projects_without_dob) > 0:
    print("\nHead of unmatched Multifamily Finance Program new construction projects:")
    hpd_multifamily_finance_new_construction_unmatched_projects_df = hpd_multifamily_finance_new_construction_for_matching_df[hpd_multifamily_finance_new_construction_for_matching_df['Project ID'].isin(mfp_projects_without_dob)].copy()
    # Ensure BBL is displayed as a string, not float
    if 'BBL' in hpd_multifamily_finance_new_construction_unmatched_projects_df.columns:
        hpd_multifamily_finance_new_construction_unmatched_projects_df['BBL'] = hpd_multifamily_finance_new_construction_unmatched_projects_df['BBL'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    excluded_cols = [
        "Lot Area", "Available Units", "Privately Financed Units", "Extremely Low Income Units",
        "Very Low Income Units", "Low Income Units", "Moderate Income Units", "Middle Income Units",
        "Studio Units", "One Bedroom Units", "Two Bedroom Units", "Three Bedroom Units",
        "Four Bedroom Units", "Five Bedroom Units", "Six Bedroom Units", "Unknown Bedroom Units",
    ][:15]  # Limit extra-wide tables in notebook
    display_cols = []
    for c in hpd_multifamily_finance_new_construction_unmatched_projects_df.columns:
        if c not in excluded_cols:
            display_cols.append(c)
    display(hpd_multifamily_finance_new_construction_unmatched_projects_df[display_cols].head(10))
else:
    print("\nAll Multifamily Finance Program projects matched to DOB data!")


In [ ]:
# Join HPD Multifamily Finance new construction rows to DOB data,
# first on BIN, then using BBL as a backup for rows without a BIN match.
# For each HPD row, add the earliest of DOB's 'paid', 'applied', or 'approved' date
# (whichever is earliest/present among these columns in the DOB data).


# Suppress performance warnings about DataFrame fragmentation
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

# --- Ensure all date columns in the DOB dataframe are properly converted to datetime dtype ---


# Load BISWEB and DOB NOW data
dob_bisweb = pd.read_csv('data/processed/multifamily_finance_dob_bisweb_bin.csv', low_memory=False)
dob_now_path = Path('data/processed/multifamily_finance_dob_now_bin.csv')
if dob_now_path.exists():
    dob_now = pd.read_csv(dob_now_path, low_memory=False)
else:
    dob_now = pd.DataFrame()

# Combine
dob_df = pd.concat([dob_bisweb, dob_now], ignore_index=True)
print(f'Loaded DOB data: {len(dob_bisweb)} BISWEB + {len(dob_now)} DOB NOW = {len(dob_df)} total')

# Reconstruct BBL from borough/block/lot (same as cell 17)
def reconstruct_bbl(row):
    if pd.isna(row.get('borough')) or pd.isna(row.get('block')) or pd.isna(row.get('lot')):
        return None
    borough_map = {'MANHATTAN': '1', 'BRONX': '2', 'BROOKLYN': '3', 'QUEENS': '4', 'STATEN ISLAND': '5'}
    borough_code = borough_map.get(str(row['borough']).upper(), None)
    if not borough_code:
        return None
    try:
        block_str = str(int(row['block']))
        lot_str = str(int(row['lot']))
    except (ValueError, TypeError):
        return None
    # Reconstruct: borough(1) + block(5) + lot(4) = 10 digits
    bbl_str = borough_code + block_str.zfill(5) + lot_str.zfill(4)
    return bbl_str

dob_df['bbl_reconstructed'] = dob_df.apply(reconstruct_bbl, axis=1)

# Create bbl_normalized from bbl column or reconstructed BBL
if 'bbl' in dob_df.columns:
    dob_df['bbl_normalized'] = dob_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    # Fill missing values with reconstructed BBL
    dob_df['bbl_normalized'] = dob_df['bbl_normalized'].fillna(dob_df['bbl_reconstructed'])
else:
    # Use reconstructed BBL if no bbl column exists
    dob_df['bbl_normalized'] = dob_df['bbl_reconstructed']

print(f'  - bbl_normalized non-null: {dob_df["bbl_normalized"].notna().sum()}')


# Filter to only NB applications with specific document/application numbers:
# - BISWEB: doc__ = 1 (document number 01) - filter when doc__ column exists
# - DOB NOW: application number ending with 'I1' (e.g., X00969702-I1)

# Filter BISWEB data: only filter rows that have doc__ column (BISWEB data)
# Keep DOB NOW rows (which have NaN in doc__) and BISWEB rows with doc__ = 1
if 'doc__' in dob_df.columns:
    # Keep rows where doc__ is NaN (DOB NOW) OR doc__ == 1 (BISWEB doc 01)
    dob_df = dob_df[(dob_df['doc__'].isna()) | (dob_df['doc__'] == 1)]
    print(f"Filtered BISWEB to doc__ = 1 (keeping DOB NOW): {len(dob_df)} records remaining")
# Filter DOB NOW data: job_filing_number should end with 'I1'
if 'job_filing_number' in dob_df.columns:
    # Only filter rows that have job_filing_number (DOB NOW data)
    dobnow_mask = dob_df['job_filing_number'].notna()
    if dobnow_mask.any():
        dob_df = dob_df[
            (~dobnow_mask) | (dob_df['job_filing_number'].astype(str).str.endswith('I1', na=False))
        ]
        print(f"Filtered DOB NOW to I1 suffix: {len(dob_df)} records remaining")

# Create bin_normalized column from bin__ or bin column
if 'bin_normalized' not in dob_df.columns:
    if 'bin__' in dob_df.columns:
        dob_df['bin_normalized'] = dob_df['bin__'].astype(str).str.replace('.0', '')
        print(f"Created bin_normalized from bin__")
    elif 'bin' in dob_df.columns:
        dob_df['bin_normalized'] = dob_df['bin'].astype(str).str.replace('.0', '')
        print(f"Created bin_normalized from bin")


if 'bbl_normalized' not in dob_df.columns:
    # Fallback: create bbl_normalized from bbl column if it doesn't exist
    dob_df['bbl_normalized'] = dob_df['bbl'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
else:
    # Ensure bbl_normalized is properly formatted as string (10 digits, zero-padded)
    def format_bbl(x):
        if pd.isna(x) or x is None:
            return None
        try:
            # Try to convert to int then string, then pad
            return str(int(float(x))).zfill(10)
        except (ValueError, TypeError):
            x_str = str(x).strip()
            if x_str in ('None', 'nan', ''):
                return None
            if x_str.isdigit() and len(x_str) <= 10:
                return x_str.zfill(10)
            return None
    dob_df['bbl_normalized'] = dob_df['bbl_normalized'].apply(format_bbl)

# Identify all possible milestone date columns across BISWEB and DOB NOW
possible_date_cols = [
    'pre__filing_date', 'paid', 'fully_paid', 'assigned', 'approved', 'fully_permitted',
    'filing date', 'first approved date', 'first permit date', 'approved date',
    'filing_date', 'first_permit_date', 'first_approved_date', 'approved_date',
    'latest_action_date'
]
# Find intersection with dob_df columns (case insensitive, allow aliasing)
normalized_col_map = {}
dob_col_lower_map = {col.lower().replace(" ", "_"): col for col in dob_df.columns}
date_cols_final = []
for c in possible_date_cols:
    c_norm = c.lower().replace(" ", "_")
    # Try to match to actual dob_df columns
    if c_norm in dob_col_lower_map:
        real_col = dob_col_lower_map[c_norm]
        normalized_col_map[c] = real_col
        if real_col not in date_cols_final:
            date_cols_final.append(real_col)

# Convert these columns to datetime in dob_df
print(f'Found {len(date_cols_final)} date columns: {date_cols_final}')
for col in date_cols_final:
    dob_df[col] = pd.to_datetime(dob_df[col], errors='coerce')
    non_null_count = dob_df[col].notna().sum()
    print(f'  - {col}: {non_null_count} non-null dates')

# Earliest date selector function expects all possible date columns to be datetime now
def _get_earliest_date(row, date_cols):
    """Get the earliest date and the column name that provided it."""
    date_values = []
    for col in date_cols:
        v = row.get(col, None)
        if pd.notna(v):
            # Ensure it's a datetime for proper comparison
            try:
                date_val = pd.to_datetime(v, errors='coerce')
                if pd.notna(date_val):
                    date_values.append((date_val, col))
            except:
                pass
    if date_values:
        earliest = min(date_values, key=lambda x: x[0])
        return earliest[0], earliest[1]  # Return (date, column_name)
    return pd.NaT, None

# Bin/BBL handling as previously
dob_bin_dates = dob_df.copy()
# Apply function that returns both date and source column
earliest_results = dob_bin_dates.apply(lambda row: _get_earliest_date(row, date_cols_final), axis=1)

# Extract date and source column from tuples, then get date directly from source column
# This ensures the date value exactly matches what's in the source column
# Use the index from earliest_results to match with dob_bin_dates index
earliest_dates = []
earliest_sources = []
for orig_idx in earliest_results.index:
    result = earliest_results.loc[orig_idx]
    if isinstance(result, tuple) and len(result) >= 2:
        source_col = result[1]
        row = dob_bin_dates.loc[orig_idx]
        # Get the date directly from the source column to ensure accuracy
        if source_col and source_col in row and pd.notna(row[source_col]):
            source_date = pd.to_datetime(row[source_col], errors='coerce')
            earliest_dates.append(source_date)
        else:
            # Fallback to the date from the tuple if source column not available
            earliest_dates.append(result[0])
        earliest_sources.append(source_col)
    elif isinstance(result, tuple) and len(result) == 1:
        earliest_dates.append(result[0])
        earliest_sources.append(None)
    else:
        earliest_dates.append(result if not isinstance(result, tuple) else pd.NaT)
        earliest_sources.append(None)

# Create Series with matching index
dob_bin_dates['earliest_dob_date'] = pd.Series(earliest_dates, index=earliest_results.index)
dob_bin_dates['earliest_dob_date_source'] = pd.Series(earliest_sources, index=earliest_results.index)
print(f'Extracted earliest dates: {dob_bin_dates["earliest_dob_date"].notna().sum()} non-null out of {len(dob_bin_dates)}')

# Ensure key application/filing date columns are datetime (if present)
for col in ['pre__filing_date', 'latest_action_date', 'filing_date', 'first_permit_date', 'first_approved_date', 'approved_date']:
    if col in dob_bin_dates.columns:
        dob_bin_dates[col] = pd.to_datetime(dob_bin_dates[col], errors='coerce')

def get_application_date(row):
    # Check for earliest milestone in a preferred order with all values as datetimes
    for col in ['pre__filing_date', 'latest_action_date']:
        v = row.get(col, None)
        if pd.notna(v):
            return v
    for col in ['filing_date', 'first_permit_date']:
        v = row.get(col, None)
        if pd.notna(v):
            return v
    if pd.notna(row.get('earliest_dob_date')):
        return row['earliest_dob_date']
    return pd.NaT

dob_bin_dates['application_date'] = dob_bin_dates.apply(get_application_date, axis=1)

def get_application_number(row):
    if 'job_filing_number' in row and pd.notna(row['job_filing_number']):
        return str(row['job_filing_number'])
    elif 'job__' in row and pd.notna(row['job__']):
        return str(row['job__'])
    return None

dob_bin_dates['application_number'] = dob_bin_dates.apply(get_application_number, axis=1)

dob_bin_dates_sorted = dob_bin_dates.sort_values('application_date', ascending=False, na_position='last')

# Group by BIN and get earliest date from most recent application (first row after sorting)
# After grouping, re-read the date from the source column to ensure accuracy
dob_bin_min_temp = dob_bin_dates_sorted.groupby('bin_normalized', as_index=False).first()
dob_bin_min = dob_bin_min_temp[['bin_normalized', 'earliest_dob_date', 'earliest_dob_date_source', 'application_number']].copy()

# Also get fully_permitted date if available
if 'fully_permitted' in dob_bin_min_temp.columns:
    dob_bin_min['fully_permitted_date'] = dob_bin_min_temp['fully_permitted']
print(f'After BIN groupby: {len(dob_bin_min)} unique BINs, {dob_bin_min["earliest_dob_date"].notna().sum()} with dates')

# Re-read the date from the source column to ensure it matches
for idx in dob_bin_min.index:
    source_col = dob_bin_min.loc[idx, 'earliest_dob_date_source']
    if source_col and pd.notna(source_col):
        orig_row = dob_bin_min_temp.loc[idx]
        if source_col in orig_row and pd.notna(orig_row[source_col]):
            dob_bin_min.loc[idx, 'earliest_dob_date'] = pd.to_datetime(orig_row[source_col], errors='coerce')
dob_bbl_filtered = dob_bin_dates_sorted[dob_bin_dates_sorted['bbl_normalized'].notna()].copy()
if not dob_bbl_filtered.empty:
    # Group by BBL and get earliest date from most recent application
    dob_bbl_min_temp = dob_bbl_filtered.groupby('bbl_normalized', as_index=False).first()
    dob_bbl_min = dob_bbl_min_temp[['bbl_normalized', 'earliest_dob_date', 'earliest_dob_date_source', 'application_number']].copy()
    
    # Also get fully_permitted date if available
    if 'fully_permitted' in dob_bbl_min_temp.columns:
        dob_bbl_min['fully_permitted_date'] = dob_bbl_min_temp['fully_permitted']
    
    # Re-read the date from the source column to ensure it matches
    for idx in dob_bbl_min.index:
        source_col = dob_bbl_min.loc[idx, 'earliest_dob_date_source']
        if source_col and pd.notna(source_col):
            orig_row = dob_bbl_min_temp.loc[idx]
            if source_col in orig_row and pd.notna(orig_row[source_col]):
                dob_bbl_min.loc[idx, 'earliest_dob_date'] = pd.to_datetime(orig_row[source_col], errors='coerce')
else:
    dob_bbl_min = pd.DataFrame(columns=['bbl_normalized', 'earliest_dob_date', 'earliest_dob_date_source', 'application_number'])

hpd_df = hpd_multifamily_finance_new_construction_df.copy()
hpd_df['BIN_str'] = hpd_df['BIN'].dropna().astype(str).str.replace('.0', '')
hpd_df['BBL_str'] = hpd_df['BBL'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)

# Identify placeholder BINs that should not be used for matching
placeholder_bins = ['1000000', '2000000', '3000000', '4000000', '5000000']
hpd_df['has_valid_bin'] = ~hpd_df['BIN_str'].isin(placeholder_bins)

# Only merge on BIN for rows with valid (non-placeholder) BINs
hpd_with_date = pd.merge(
    hpd_df,
    dob_bin_min.rename(columns={'bin_normalized': 'BIN_str'}),
    how='left',
    on='BIN_str'
)

# For placeholder BINs, clear out the incorrectly matched dates
hpd_with_date.loc[~hpd_with_date['has_valid_bin'], 'earliest_dob_date'] = pd.NaT
hpd_with_date.loc[~hpd_with_date['has_valid_bin'], 'earliest_dob_date_source'] = None
hpd_with_date.loc[~hpd_with_date['has_valid_bin'], 'application_number'] = None
if 'fully_permitted_date' in hpd_with_date.columns:
    hpd_with_date.loc[~hpd_with_date['has_valid_bin'], 'fully_permitted_date'] = pd.NaT
no_bin_match_mask = hpd_with_date['earliest_dob_date'].isna()
if no_bin_match_mask.any():
    to_fill = hpd_with_date.loc[no_bin_match_mask, ['BBL_str']]
    filled = pd.merge(
        to_fill,
        dob_bbl_min.rename(columns={'bbl_normalized': 'BBL_str'}),
        how='left',
        on='BBL_str'
    )
    hpd_with_date.loc[no_bin_match_mask, 'earliest_dob_date'] = filled['earliest_dob_date'].values
    if 'application_number' in filled.columns:
        hpd_with_date.loc[no_bin_match_mask, 'application_number'] = filled['application_number'].values
    if 'earliest_dob_date_source' in filled.columns:
        hpd_with_date.loc[no_bin_match_mask, 'earliest_dob_date_source'] = filled['earliest_dob_date_source'].values
    if 'fully_permitted_date' in filled.columns:
        hpd_with_date.loc[no_bin_match_mask, 'fully_permitted_date'] = filled['fully_permitted_date'].values

hpd_multifamily_finance_new_construction_with_dob_date_df = hpd_with_date

# Use display instead of print, and always show Building ID, date source, and application number
display_cols = ['Building ID', 'Project ID', 'BIN', 'BBL', 'earliest_dob_date']
if 'earliest_dob_date_source' in hpd_multifamily_finance_new_construction_with_dob_date_df.columns:
    display_cols.append('earliest_dob_date_source')
if 'application_number' in hpd_multifamily_finance_new_construction_with_dob_date_df.columns:
    display_cols.append('application_number')
if 'fully_permitted_date' in hpd_multifamily_finance_new_construction_with_dob_date_df.columns:
    display_cols.append('fully_permitted_date')
display(hpd_multifamily_finance_new_construction_with_dob_date_df[display_cols].head(10))

num_missing_earliest_dob_date = hpd_multifamily_finance_new_construction_with_dob_date_df['earliest_dob_date'].isna().sum()
print(f"\nCount of HPD MFP new construction rows without an earliest DOB milestone date: {num_missing_earliest_dob_date}")

no_dob_milestone_df = hpd_multifamily_finance_new_construction_with_dob_date_df[
    hpd_multifamily_finance_new_construction_with_dob_date_df['earliest_dob_date'].isna()
]

# Display the full table of rows without a DOB milestone date: include Building ID
no_dob_milestone_table = no_dob_milestone_df[['Building ID', 'Project ID', 'Project Name', 'BIN', 'BBL', 'earliest_dob_date']]
display(no_dob_milestone_table)
print(f"Total without DOB milestone date: {len(no_dob_milestone_table)}")

def bbl_lot_starts_with_75(bbl):
    if pd.isna(bbl):
        return False
    bbl_str = str(int(float(bbl))).zfill(10)
    lot_str = bbl_str[-4:]
    return lot_str.startswith('75')

lots_start_75_mask = no_dob_milestone_df['BBL'].apply(bbl_lot_starts_with_75)
num_lots_start_75 = lots_start_75_mask.sum()
print(f"\nHow many have BBL lot numbers starting with '75': {num_lots_start_75}")

null_bin_bbl_mask = no_dob_milestone_df['BIN'].isna() & no_dob_milestone_df['BBL'].isna()
null_bin_bbl_df = no_dob_milestone_df[null_bin_bbl_mask]

full_df = hpd_multifamily_finance_new_construction_with_dob_date_df
duplicated_project_ids_in_full = full_df['Project ID'][full_df['Project ID'].duplicated(keep=False)].unique()

null_bin_bbl_with_duplicated_projid = null_bin_bbl_df['Project ID'].isin(duplicated_project_ids_in_full)
count_null_bin_bbl_and_duplicated = null_bin_bbl_with_duplicated_projid.sum()
print(f"\nHow many have both BIN and BBL null and with a duplicated Project ID in FULL dataset: {count_null_bin_bbl_and_duplicated}")

null_bin_bbl_duplicated_mask = null_bin_bbl_df['Project ID'].duplicated(keep=False)
count_null_bin_bbl_duplicated_in_subset = null_bin_bbl_duplicated_mask.sum()
print(f"How many have both BIN and BBL null and Project ID duplicated within null subset: {count_null_bin_bbl_duplicated_in_subset}")

if count_null_bin_bbl_and_duplicated > 0:
    print("\nExamples of rows with both BIN and BBL null where Project ID appears multiple times in FULL dataset:")
    display(null_bin_bbl_df[null_bin_bbl_with_duplicated_projid][['Building ID', 'Project ID', 'Project Name', 'BIN', 'BBL']])
    
    print("\nAll rows for these projects in FULL dataset (for context):")
    projects_to_show = null_bin_bbl_df[null_bin_bbl_with_duplicated_projid]['Project ID'].unique()
    all_rows_for_projects = full_df[full_df['Project ID'].isin(projects_to_show)]
    display_cols = ['Building ID', 'Project ID', 'Project Name', 'BIN', 'BBL', 'earliest_dob_date']
    display_cols = [col for col in display_cols if col in all_rows_for_projects.columns]
    display(all_rows_for_projects[display_cols].sort_values('Project ID'))

if count_null_bin_bbl_duplicated_in_subset > 0:
    print("\nExamples of rows with both BIN and BBL null and duplicated Project IDs (within null subset):")
    display(null_bin_bbl_df[null_bin_bbl_duplicated_mask][['Building ID', 'Project ID', 'Project Name', 'BIN', 'BBL']])


# Export to CSV
output_path = 'output/hpd_multifamily_finance_new_construction_with_dob_date.csv'
os.makedirs('output', exist_ok=True)
hpd_multifamily_finance_new_construction_with_dob_date_df.to_csv(output_path, index=False)
print(f"\nExported to {output_path}")
print(f"  Total rows: {len(hpd_multifamily_finance_new_construction_with_dob_date_df)}")
print(f"  Rows with DOB dates: {hpd_multifamily_finance_new_construction_with_dob_date_df['earliest_dob_date'].notna().sum()}")
print(f"  Rows without DOB dates: {hpd_multifamily_finance_new_construction_with_dob_date_df['earliest_dob_date'].isna().sum()}")

## 🏛️ Step 4: Query Certificate of Occupancy


In [ ]:
# Step 3B Configuration
print("\n" + "=" * 70)
print("STEP 3B: QUERY CERTIFICATE OF OCCUPANCY")
print("=" * 70)

# Extract BINs from hpd_multifamily_finance_new_construction_df (in memory from Step 3A)
bins_list = _extract_bins_from_df(hpd_multifamily_finance_new_construction_df)
print(f"\n📋 Extracted {len(bins_list)} BINs from filtered dataset")

# First, query by BIN only to see which BINs are found
print(f"🏛️ Querying CO APIs by BIN first to identify unfound BINs...")
co_filings_df = _query_co_filings_from_bins_and_bbls(bins_list, bbl_list=None, output_path=None)

# Identify which BINs were found in CO data
found_bins = set()
if co_filings_df is not None and not co_filings_df.empty:
    if 'bin_normalized' in co_filings_df.columns:
        found_bins = set(co_filings_df['bin_normalized'].dropna().astype(str).unique())
    elif 'bin' in co_filings_df.columns:
        found_bins = set(co_filings_df['bin'].dropna().astype(str).unique())
    elif 'bin_number' in co_filings_df.columns:
        found_bins = set(co_filings_df['bin_number'].dropna().astype(str).unique())

print(f"\n📊 BIN query results: {len(found_bins)} unique BINs found in CO data out of {len(bins_list)} queried")

# Extract BBLs for fallback querying (placeholder BINs + BINs not found in CO data)
bbls_to_query = []
placeholder_bins = ['1000000', '2000000', '3000000', '4000000', '5000000']

for _, row in hpd_multifamily_finance_new_construction_df.iterrows():
    bin_val = row.get('BIN')
    bin_str = str(bin_val).replace('.0', '') if pd.notna(bin_val) else ''
    bbl_val = row.get('BBL')
    
    # Include BBL if:
    # (1) BIN is placeholder/invalid, OR
    # (2) BIN was not found in CO data, OR
    # (3) No BIN but has BBL
    should_query_bbl = (
        (bin_str in placeholder_bins) or
        (bin_str and bin_str not in found_bins) or
        (pd.isna(bin_val))
    )
    
    if should_query_bbl and pd.notna(bbl_val):
        try:
            bbl_str = str(int(float(bbl_val))).zfill(10)
            bbls_to_query.append(bbl_str)
        except:
            pass

bbls_to_query = sorted(list(set(bbls_to_query)))
print(f"📋 Extracted {len(bbls_to_query)} BBLs for fallback querying")
print(f"   - Placeholder BINs: {sum(1 for _, row in hpd_multifamily_finance_new_construction_df.iterrows() if str(row.get('BIN', '')).replace('.0', '') in placeholder_bins and pd.notna(row.get('BBL')))}")
print(f"   - BINs not found in CO data: {sum(1 for _, row in hpd_multifamily_finance_new_construction_df.iterrows() if str(row.get('BIN', '')).replace('.0', '') not in found_bins and str(row.get('BIN', '')).replace('.0', '') not in placeholder_bins and pd.notna(row.get('BIN')) and pd.notna(row.get('BBL')))}")
print(f"   - Missing BINs: {sum(1 for _, row in hpd_multifamily_finance_new_construction_df.iterrows() if pd.isna(row.get('BIN')) and pd.notna(row.get('BBL')))}")

# Query by BBL for unfound records
if len(bbls_to_query) > 0:
    print(f"\n🏛️ Querying CO APIs by BBL for {len(bbls_to_query)} BBLs...")
    co_filings_bbl_df = _query_co_filings_from_bins_and_bbls(bin_list=[], bbl_list=bbls_to_query, output_path=None)
    
    # Combine BIN and BBL results
    if co_filings_bbl_df is not None and not co_filings_bbl_df.empty:
        if co_filings_df is not None and not co_filings_df.empty:
            # Combine both dataframes
            all_cols = list(set(co_filings_df.columns.tolist() + co_filings_bbl_df.columns.tolist()))
            co_filings_df_aligned = co_filings_df.reindex(columns=all_cols)
            co_filings_bbl_df_aligned = co_filings_bbl_df.reindex(columns=all_cols)
            co_filings_df = pd.concat([co_filings_df_aligned, co_filings_bbl_df_aligned], ignore_index=True)
        else:
            co_filings_df = co_filings_bbl_df
else:
    print(f"\n✓ All BINs found in CO data, no BBL fallback needed")

# Display CO data if available
if co_filings_df is not None:
    print(f"📊 Certificate of Occupancy Data: {co_filings_df.shape[0]} records")
    print("Columns:")
    for col in co_filings_df.columns:
        print(f"  - {col}")
    
    print("\n📊 Sample CO Data:")
    display(co_filings_df.head())
    
    # Show some statistics
    if "issue_date" in co_filings_df.columns:
        print("\n📈 CO Issue Date Statistics:")
        display(co_filings_df["issue_date"].describe())
else:
    print("⚠️ No CO data available")

In [ ]:
# Join CO data with HPD data to get earliest CO date
print("\n" + "=" * 70)
print("JOINING CO DATA WITH HPD DATA")
print("=" * 70)

if co_filings_df is not None and not co_filings_df.empty:
    # Identify CO date columns
    co_date_cols = []
    for col in co_filings_df.columns:
        if 'date' in col.lower() and 'issue' in col.lower():
            co_date_cols.append(col)
    
    print(f"\n📅 CO date columns found: {co_date_cols}")
    
    # Convert date columns to datetime
    for col in co_date_cols:
        co_filings_df[col] = pd.to_datetime(co_filings_df[col], errors='coerce')
    
    # Create normalized BIN and BBL columns for joining
    if 'bin_normalized' not in co_filings_df.columns:
        if 'bin' in co_filings_df.columns:
            co_filings_df['bin_normalized'] = co_filings_df['bin'].astype(str).str.replace('.0', '', regex=False)
        elif 'bin_number' in co_filings_df.columns:
            co_filings_df['bin_normalized'] = co_filings_df['bin_number'].astype(str).str.replace('.0', '', regex=False)
    
    if 'bbl_normalized' not in co_filings_df.columns:
        if 'bbl' in co_filings_df.columns:
            co_filings_df['bbl_normalized'] = co_filings_df['bbl'].astype(str).str.zfill(10)
    
    # Get earliest CO date for each BIN
    def get_earliest_co_date(row, date_cols):
        dates = [row[col] for col in date_cols if col in row and pd.notna(row[col])]
        if dates:
            return min(dates)
        return pd.NaT
    
    co_filings_df['earliest_co_date'] = co_filings_df.apply(lambda row: get_earliest_co_date(row, co_date_cols), axis=1)
    
    # Group by BIN to get earliest CO date per BIN
    co_bin_min = co_filings_df[co_filings_df['bin_normalized'].notna()].groupby('bin_normalized', as_index=False)['earliest_co_date'].min()
    print(f"\n📊 CO dates by BIN: {len(co_bin_min)} unique BINs with CO dates")
    
    # Group by BBL to get earliest CO date per BBL (for fallback)
    co_bbl_min = pd.DataFrame()
    if 'bbl_normalized' in co_filings_df.columns:
        co_bbl_min = co_filings_df[co_filings_df['bbl_normalized'].notna()].groupby('bbl_normalized', as_index=False)['earliest_co_date'].min()
        print(f"📊 CO dates by BBL: {len(co_bbl_min)} unique BBLs with CO dates")
    
    # Join with HPD data (use the dataframe with DOB dates)
    hpd_with_co = hpd_multifamily_finance_new_construction_with_dob_date_df.copy()
    
    # Drop earliest_co_date if it already exists (from previous run)
    if 'earliest_co_date' in hpd_with_co.columns:
        hpd_with_co = hpd_with_co.drop(columns=['earliest_co_date'])
    
    # Create normalized BIN and BBL in HPD data
    hpd_with_co['BIN_str'] = hpd_with_co['BIN'].astype(str).str.replace('.0', '', regex=False)
    hpd_with_co['BBL_str'] = hpd_with_co['BBL'].apply(lambda x: str(int(float(x))).zfill(10) if pd.notna(x) else None)
    
    # Merge on BIN first
    if not co_bin_min.empty:
        hpd_with_co = pd.merge(
            hpd_with_co,
            co_bin_min.rename(columns={'bin_normalized': 'BIN_str'}),
            how='left',
            on='BIN_str'
        )
    else:
        # If no BIN matches, create empty column
        hpd_with_co['earliest_co_date'] = pd.NaT
    
    # BBL fallback for rows without CO date
    no_co_mask = hpd_with_co['earliest_co_date'].isna() if 'earliest_co_date' in hpd_with_co.columns else pd.Series([True] * len(hpd_with_co))
    if no_co_mask.any() and not co_bbl_min.empty:
        to_fill = hpd_with_co.loc[no_co_mask, ['BBL_str']]
        filled = pd.merge(
            to_fill,
            co_bbl_min.rename(columns={'bbl_normalized': 'BBL_str'}),
            how='left',
            on='BBL_str'
        )
        if 'earliest_co_date' in filled.columns:
            hpd_with_co.loc[no_co_mask, 'earliest_co_date'] = filled['earliest_co_date'].values
    
    # Update the dataframe
    hpd_multifamily_finance_new_construction_with_dob_date_df = hpd_with_co
    
    # Display results
    print(f"\n📊 Results:")
    print(f"   Total HPD rows: {len(hpd_with_co)}")
    print(f"   Rows with CO dates: {hpd_with_co['earliest_co_date'].notna().sum()}")
    print(f"   Rows without CO dates: {hpd_with_co['earliest_co_date'].isna().sum()}")
    
    # Display sample
    display_cols = ['Building ID', 'Project ID', 'BIN', 'BBL', 'earliest_dob_date', 'fully_permitted_date', 'earliest_co_date']
    display_cols = [col for col in display_cols if col in hpd_with_co.columns]
    print(f"\n📋 Sample data:")
    display(hpd_with_co[display_cols].head(10))
    
    # Export the final dataframe
    import os
    os.makedirs("output", exist_ok=True)
    hpd_with_co.to_csv("output/hpd_multifamily_finance_new_construction_with_all_dates.csv", index=False)
    print("\n✅ Exported HPD + DOB/CO matched data to output/hpd_multifamily_finance_new_construction_with_all_dates.csv")
else:
    print("\n⚠️ No CO data available to join")
